# Schreier-Sims for Rubik's Cubes

In [1]:
import numpy as np, time, random
from sympy.combinatorics import Permutation
from functions import enumerate_cube, get_human_order, get_human_order_multiple, get_solved_states, compute_schreiers_tree, compute_generating_set, apply_sims_filter
import config

# choose size of Rubik's cube
n = 3

# N: order of the symmetric group (number of enumerated stickers)
if n % 2 == 0:
    N = 6 * n**2
else:
    N = 6 * n**2 - 6

# set of moves that generate the Rubik's cube
generators = enumerate_cube(n, N)

identity_order = range(N)
random_order = random.sample(range(N), N)
human_order = get_human_order(n)
human_oder_multiple = get_human_order_multiple(n)
# choose solving order
solving_order = [(1,3,5,7),(0,2,4,6),(9,13,35,39),(25,27,29,31),(24,26,28,30)]

# list of group orders of the stabilizer groups
orders_list = []

# list of generating sets of the stabilizer groups
generators_list = []
length_generators_list_before_sims_filter = []
length_generators_list = []

# list of Schreiers vectors for the stabilizer groups
schreiers_tree_list = []

# dictionary of lengths of moves written as composition of original generators
config.lengths = {Permutation(N-1):0}
for generator in generators:
    config.lengths[generator] = 1

# the algorithm
i = 0

length_generators_list_before_sims_filter.append(len(generators))

temp_generators = apply_sims_filter(generators, N)
length_generators_list.append(len(temp_generators))
generators_list.append(temp_generators)

start_time = time.time()

while len(temp_generators) > 0:
    k = solving_order[i]

    schreiers_tree = compute_schreiers_tree(temp_generators, N, k)
    schreiers_tree_list.append(schreiers_tree)

    orders_list.append(len(schreiers_tree))

    new_generators = compute_generating_set(temp_generators, schreiers_tree, k)
    length_generators_list_before_sims_filter.append(len(new_generators))

    temp_generators = apply_sims_filter(new_generators, N)
    length_generators_list.append(len(temp_generators))
    generators_list.append(temp_generators)

    i += 1

    elapsed_time = time.time()-start_time
    print(f"Time after iteration {i}: {elapsed_time:.2f} seconds")


# compute visually solved states (useful for membership testing)
solved_states = get_solved_states(n, N)

Time after iteration 1: 428.64 seconds
Time after iteration 2: 11532.86 seconds
Time after iteration 3: 11655.55 seconds
Time after iteration 4: 11655.88 seconds
Time after iteration 5: 11656.26 seconds


## Order of Rubik's cube group

In [2]:
orders_list = np.array(orders_list, dtype=object)
print(np.prod(orders_list))

43252003274489856000


## Membership testing and length of solving algorithm

In [8]:
# choose the element you want to perform the check on

# some examples of moves:

# for generator in generators:
#     move = Permutation(generator)
#     print(generator)
l = list(generators)
move = l[1]*l[2]*l[3]*(l[2]**-1)*l[5]*l[1]*(l[3]**-1)*l[0]*l[2]*l[1]
print(move)
# move = Permutation(N-1)(0,10,20)(6,12,40) # double corner twist on 3x3
# move = Permutation(N-1)(51,63)(0,40,20)(32,52,72) # double corner twist and enter swap on 4x4

# the algorithm

contained = False
length_solving_algorithm = 0

for solved_state in solved_states:
    g = move * solved_state
    length_solving_algorithm = 0

    i = 0
    while i < len(schreiers_tree_list):
        k = solving_order[i]

        schreiers_tree = schreiers_tree_list[i]
        h = tuple(g(x) for x in k)
        u = schreiers_tree.get(h)
        if u is None:
            break
        else:
            g = g * u**-1
            length_solving_algorithm += config.lengths[u]
        i += 1

    if g == Permutation(N-1):
        contained = True
        print(length_solving_algorithm)
        break

print(contained)

(0 8 10 28 20 22)(1 25 39)(2 44 6 32 16 46 34 24 12 4 36 14 18 38 40 42 26 30)(3 27 29 21 7 47 35 33 23 15 9 11 13 17)(19 37 43)(31 45)
6238
True
